In [1]:
 import pandas as pd
 import mysql.connector 

 host = '127.0.0.1' 
 user = 'root' 
 password = '' 
 database = 'ordermanagementdb' 

In [2]:
 connection = mysql.connector.connect( 
 host=host, 
 user=user, 
 password=password, 
 database=database 
 )  
 customers = pd.read_sql('SELECT * FROM customers', connection) 
 products = pd.read_sql('SELECT * FROM products', connection) 
 orders = pd.read_sql('SELECT * FROM orders', connection) 
 orderitems = pd.read_sql('SELECT * FROM orderitems', connection) 
 #connection.close()
 print(customers)
 print(products)
 print(orders)
 print(orderitems)


    CustomerID FirstName    LastName                         Email     Phone  \
0            1      Amit      Sharma       amit.sharma@example.com  555-0101   
1            2    Rajesh       Kumar      rajesh.kumar@example.com  555-0102   
2            3    Vikram       Singh      vikram.singh@example.com  555-0103   
3            4    Sunita         Rai        sunita.rai@example.com  555-0104   
4            5     Anita       Gupta       anita.gupta@example.com  555-0105   
..         ...       ...         ...                           ...       ...   
95          96    Monika      Duggal     monika.duggal@example.com  555-0196   
96          97      Ravi       Bajaj        ravi.bajaj@example.com  555-0197   
97          98     Nisha  Chaturvedi  nisha.chaturvedi@example.com  555-0198   
98          99     Tarun       Goyal       tarun.goyal@example.com  555-0199   
99         100     Kavya        Soni        kavya.soni@example.com  555-0200   

        Address   City  State ZipCode  

C:\Users\tejas jaiswal\AppData\Local\Temp\ipykernel_19660\1664400320.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql('SELECT * FROM customers', connection)
C:\Users\tejas jaiswal\AppData\Local\Temp\ipykernel_19660\1664400320.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql('SELECT * FROM products', connection)
C:\Users\tejas jaiswal\AppData\Local\Temp\ipykernel_19660\1664400320.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql('SELECT * FROM orders

In [4]:
 print(customers.info())
 print(products.info())
 print(orders.info())
 print(orderitems.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  100 non-null    int64 
 1   FirstName   100 non-null    object
 2   LastName    100 non-null    object
 3   Email       100 non-null    object
 4   Phone       100 non-null    object
 5   Address     100 non-null    object
 6   City        100 non-null    object
 7   State       100 non-null    object
 8   ZipCode     100 non-null    object
dtypes: int64(1), object(8)
memory usage: 7.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    50 non-null     int64  
 1   ProductName  50 non-null     object 
 2   Category     50 non-null     object 
 3   Price        50 non-null     float64
 4   Stock        50 non-null     int64  
dtypes

In [3]:
# Handle missing values
customers.fillna('', inplace=True)
orders.fillna('', inplace=True)
products.fillna('', inplace=True)
orderitems.fillna('', inplace=True)

# Ensure correct data types
customers = customers.astype({'CustomerID': 'int64', 'FirstName': 'object', 'LastName': 'object', 'Email': 'object', 'Phone': 'object', 'Address': 'object', 'City': 'object', 'State': 'object', 'ZipCode': 'object'})
orders = orders.astype({'OrderID': 'int64', 'OrderDate': 'datetime64[ns]', 'CustomerID': 'int64'})
products = products.astype({'ProductID': 'int64', 'ProductName': 'object', 'Category': 'object', 'Price': 'float64', 'Stock': 'int64'})
orderitems = orderitems.astype({'OrderItemID': 'int64', 'OrderID': 'int64', 'ProductID': 'int64', 'Quantity': 'int64', 'Price': 'float64'})



In [5]:
# Calculate total amount spent by each customer
orderitems['TotalPrice'] = orderitems['Quantity'] * orderitems['Price']
customer_spending = orderitems.groupby('OrderID')['TotalPrice'].sum().reset_index()
customer_orders = orders.merge(customer_spending, on='OrderID')
customer_summary = customer_orders.groupby('CustomerID').agg({
    'OrderID': 'count',
    'TotalPrice': 'sum'
}).rename(columns={'OrderID': 'OrderCount', 'TotalPrice': 'TotalSpent'}).reset_index()

# Segment customers based on their purchasing behavior
def segment_customer(row):
    if row['OrderCount'] > 10 and row['TotalSpent'] > 1000:
        return 'High Value'
    elif row['OrderCount'] > 5:
        return 'Medium Value'
    else:
        return 'Low Value'

customer_summary['Segment'] = customer_summary.apply(segment_customer, axis=1)

# Display customer segments
print(customer_summary)
customer_summary.to_excel('customer_summary.xlsx',index=False)

    CustomerID  OrderCount  TotalSpent       Segment
0            1           2      425.49     Low Value
1            2           4     5118.36     Low Value
2            3           1      216.40     Low Value
3            4           3     2184.87     Low Value
4            5           3     2007.19     Low Value
..         ...         ...         ...           ...
94          96           5     2647.87     Low Value
95          97           5     2962.03     Low Value
96          98           5     2258.96     Low Value
97          99           7     6042.43  Medium Value
98         100           8     6853.08  Medium Value

[99 rows x 4 columns]


In [12]:
from sklearn.linear_model import LinearRegression
import numpy as np

def predict_future_sales(orders, orderitems):
    # Prepare the data
    orderitems['TotalPrice'] = orderitems['Quantity'] * orderitems['Price']
    sales_data = orders.merge(orderitems.groupby('OrderID')['TotalPrice'].sum().reset_index(), on='OrderID')
    sales_data['OrderDate'] = pd.to_datetime(sales_data['OrderDate'])
    sales_data['OrderYearMonth'] = sales_data['OrderDate'].dt.to_period('M')
    monthly_sales = sales_data.groupby('OrderYearMonth')['TotalPrice'].sum().reset_index()
    
    # Convert period to datetime for regression analysis
    monthly_sales['OrderYearMonth'] = monthly_sales['OrderYearMonth'].dt.to_timestamp()
    
    # Prepare the features and target variable
    X = np.array(range(len(monthly_sales))).reshape(-1, 1)
    y = monthly_sales['TotalPrice'].values

    # Train the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict future sales (next 6 months)
    future_months = np.array(range(len(monthly_sales), len(monthly_sales) + 6)).reshape(-1, 1)
    future_sales = model.predict(future_months)
    
    # Return the predicted sales
    return future_sales

# Predict future sales
future_sales = predict_future_sales(orders, orderitems)
print("Future Sales (next 6 months):", future_sales)


Future Sales (next 6 months): [26915.29333333 26808.87679487 26702.46025641 26596.04371795
 26489.62717949 26383.21064103]


In [6]:
customers.to_excel('customers.xlsx',index=False)
products.to_excel('products.xlsx',index=False)
orders.to_excel('orders.xlsx',index=False)
orderitems.to_excel('orderitems.xlsx',index=False)